#### 0.数据探索部分
这个笔记本展示了如何对./data/raw中的原始数据集进行预处理，最终生成可供建模的数据集，保存在./data/processed中。

In [1]:
import pandas as pd

初步分析阶段先将字段以字符串形式录入，便于后期统一格式

In [2]:
BigData = pd.read_csv("../data/raw/浙江大学-图书外借数据-2013~2018.csv", encoding='utf8', dtype=str)
BigData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067948 entries, 0 to 2067947
Data columns (total 27 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   UNIVERSITY_ID      object
 1   ITEM_ID            object
 2   SUBLIBRARY         object
 3   LOAN_DATE          object
 4   LOAN_HOUR          object
 5   DUE_DATE           object
 6   DUE_HOUR           object
 7   RETURNED_DATE      object
 8   RETURNED_HOUR      object
 9   RETURNED_LOCATION  object
 10  ITEM_STATUS        object
 11  RENEWAL_NO         object
 12  LASTRENEW_DATE     object
 13  RECALL_DATE        object
 14  RECALL_DUE_DATE    object
 15  ITEM_CALLNO        object
 16  PUBLISH_YEAR       object
 17  AUTHOR             object
 18  TITLE              object
 19  PRESS              object
 20  ISBN               object
 21  HOLD_DAYS          object
 22  OVERDUE_DAYS       object
 23  PATRON_ID          object
 24  STUDENT_GRADE      object
 25  PATRON_DEPT        object
 26  PATRON_TYPE   

In [3]:
SmallData = pd.read_csv("../data/raw/浙江大学-图书外借数据-2019.csv", encoding='utf8', dtype=str)
SmallData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192701 entries, 0 to 192700
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   UNIVERSITY_ID    192701 non-null  object
 1   ITEM_ID          192701 non-null  object
 2   SUBLIBRARY       192701 non-null  object
 3   LOAN_DATE        192701 non-null  object
 4   LOAN_HOUR        192701 non-null  object
 5   DUE_DATE         192701 non-null  object
 6   DUE_HOUR         192701 non-null  object
 7   RETURNED_DATE    192701 non-null  object
 8   RETURNED_HOUR    192701 non-null  object
 9   ITEM_STATUS      192701 non-null  object
 10  RENEWAL_NO       192701 non-null  object
 11  LASTRENEW_DATE   192701 non-null  object
 12  RECALL_DATE      192701 non-null  object
 13  RECALL_DUE_DATE  192701 non-null  object
 14  ITEM_CALLNO      192696 non-null  object
 15  PUBLISH_YEAR     192701 non-null  object
 16  AUTHOR           192409 non-null  object
 17  TITLE     

In [4]:
BigData.sample(3)

,UNIVERSITY_ID,ITEM_ID,SUBLIBRARY,LOAN_DATE,LOAN_HOUR,DUE_DATE,DUE_HOUR,RETURNED_DATE,RETURNED_HOUR,RETURNED_LOCATION,...,AUTHOR,TITLE,PRESS,ISBN,HOLD_DAYS,OVERDUE_DAYS,PATRON_ID,STUDENT_GRADE,PATRON_DEPT,PATRON_TYPE
1387373,10335,001349610000010,MLT,20140429,1524,20140608,2150,20140529,1001,NaN,...,达尔文,物种起源,商务印书馆,978-7-100-07775-0,30,-10,d6e76443d0d8088752f8c3a8f8792bd7,2013,信息与电子工程学院,全日制硕士生
449602,10335,000487719000090,TYB,20180716,1441,20180917,1441,20181114,1012,NaN,...,梁衡,数理化通俗演义,电子工业出版社,NaN,121,58,9bec029803d39228f946ee70154200e4,2008,图书馆,事业编制人员
1085988,10335,000091459000050,BHB,20170704,1644,20170919,2359,20170705,1306,NaN,...,横沟正史,女王蜂,珠海出版社,7-80689-037-8,1,-76,8e151209490929d8cc3bdf5fdd1d408f,2016,高分子科学与工程学系,全日制本科生


下面是构建模型所需要的数据列

In [5]:
user_columns = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_columns = ['ITEM_ID', 'SUBLIBRARY', 'ITEM_CALLNO', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         图书索书号        出版年          作者       题目    出版社
time = 'LOAN_DATE'

将两份数据集合并

In [6]:
UsedColumns = user_columns + item_columns + [time]
UsedData = pd.concat([BigData[UsedColumns], SmallData[UsedColumns]])

In [7]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 12 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   ITEM_CALLNO    object
 7   PUBLISH_YEAR   object
 8   AUTHOR         object
 9   TITLE          object
 10  PRESS          object
 11  LOAN_DATE      object
dtypes: object(12)
memory usage: 224.2+ MB


In [8]:
UsedData.sample(3)

,PATRON_ID,STUDENT_GRADE,PATRON_DEPT,PATRON_TYPE,ITEM_ID,SUBLIBRARY,ITEM_CALLNO,PUBLISH_YEAR,AUTHOR,TITLE,PRESS,LOAN_DATE
1563924,9f1d81741bf230924120df0790254462,2012,人文学院,全日制博士生,000752156000020,SLT,C913.3/CZ9,2009,"张建,",休闲都市论,东方出版中心,20131216
107215,d77a98a0f5a779669a41416f6b32fda2,2014,农业与生物技术学院,全日制硕士生,001054912000050,BHB,I565.447/CM2.10,2008,莫泊桑,莫泊桑中短篇小说选,华夏出版社,20161016
565077,d17ae1c63931140f184509c9b97fb00f,2016,控制科学与工程学院,全日制本科生,001570884000030,BLT,I712.457/CP5.3,2015,"普罗克斯,",船讯,人民文学出版社,20180127


本项目将借阅记录中的空值字段采用相同值’na'进行填充，特征处理时将其作为空特征。对于图书类别，将其索书号按/分割后得到图书大类CALLNO1与小类CALLNO2，将借阅日期转为整形，便于之后比较大小。

In [9]:
import re

def get_Bletter(str0):   # 取出大写字母
    b = re.sub(u"([^\u0041-\u007a])", "", str0)
    return b

UsedData = UsedData.fillna(value='na')
UsedData['CALLNO1'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[0].map(lambda x: get_Bletter(str(x)))
UsedData['CALLNO2'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[1].map(lambda x: get_Bletter(str(x)))
UsedData[time]=UsedData[time].astype(int)
UsedData = UsedData.drop(columns='ITEM_CALLNO')  # 删掉不再需要的列x

此时可以看到UsedData中的空值消失，并且多出了两个类别列

In [10]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


将数据存入./data/processed中，作为后续分析数据

In [11]:
import os

UsedData.to_csv("../data/processed/ZJULibrary2013_2019.csv")

可以再读取下来观察一下是否与之前的处理结果一样

In [15]:
Data = pd.read_csv("../data/processed/ZJULibrary2013_2019.csv", index_col = 0, dtype= str)
Data[time]=Data[time].astype(int)
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


至此完成了原始数据集的合并，填补了数据集中的空值，得到了图书的类别数据，最后将数据重新写入./data/processed/ZJULibrary2013_2019.csv中。